In [ ]:
#compute optimal thershold sex

In [2]:
import os
import sys
import json
import copy
import socket
import getpass
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
sys.path.append('../')
import pickle
import warnings
from sklearn.metrics import recall_score, matthews_corrcoef, roc_auc_score, f1_score
from collections import defaultdict
import json

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def flatten(l):
    return [item for sublist in l for item in sublist]

def compute_opt_thres(target, pred):
    opt_thres = 0
    opt_f1 = 0
    for i in np.arange(0.001, 0.999, 0.001):
        f1 = f1_score(target, pred >= i)
        if f1 >= opt_f1:
            opt_thres = i
            opt_f1 = f1
    return opt_thres

In [3]:
#root_dir = Path(f'/path/to/your/root')
root_dir = Path(f'/home/lchanch/model_training/ISNetDANN/train/grid_sex_ISNetDANN')

In [4]:
if Path('opt_thres.pkl').is_file():
    already_computed = set(pickle.load(Path('opt_thres.pkl').open('rb')).keys())
else:
    already_computed = set()

In [5]:
results = []
for i in tqdm(root_dir.glob('**/done')):
    args = json.load((i.parent/'args.json').open('r'))
    if (args['dataset'][0], args['task'], args['attr'], args['algorithm']) in already_computed:
        continue
    
    final_res = pickle.load((i.parent/'final_results.pkl').open('rb'))
    
    ssets = ['va', 'te']
  
    for sset in ssets:
        if sset in final_res:
            args[f'{sset}_auroc'] = final_res[sset]['overall']['AUROC']
            if sset == 'va':
                args[f'{sset}_min_attr_auroc'] = final_res[sset]['min_attr']['AUROC']
    args['va_y'] = final_res['va']['y']
    args['va_preds'] = final_res['va']['preds']
    
    results.append(args)
df = pd.DataFrame(results)

32it [00:00, 215.04it/s]


In [7]:
df['dataset'] = df['dataset'].apply(lambda x: x[0])

In [8]:
df.shape

(32, 33)

In [9]:
df.head()

,store_name,dataset,task,attr,group_def,algorithm,output_dir,data_dir,hparams,hparams_seed,...,heat,use_masks,loadCuts,tuneCut,only_tune_cut,va_auroc,va_min_attr_auroc,te_auroc,va_y,va_preds
0,b27dd33301c5fb5687a02dd030985e82,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},0,...,False,True,False,False,False,0.798700,0.785944,0.800049,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.4653238, 0.43727994, 0.05265071, 0.14978917..."
1,93e278b999e2ea71f37c78e8fc6f5548,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},7,...,False,True,False,False,False,0.825635,0.810588,0.828288,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.56875765, 0.777, 0.061805256, 0.0927816, 0...."
2,e81ec22bafc012c649bcd6d37a78bb30,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},6,...,False,True,False,False,False,0.819142,0.804570,0.823059,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.5392749, 0.65393984, 0.08238168, 0.09234505..."
3,694785729bdc1d467d7badbb54b98b56,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},6,...,False,True,False,False,False,0.825064,0.810579,0.828439,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.6038705, 0.8439603, 0.06819831, 0.07568497,..."
4,e26c7dfc2d5a55375aa5cb565b50bd31,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},3,...,False,True,False,False,False,0.790151,0.778468,0.793279,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.48758438, 0.43101794, 0.039706927, 0.162422..."


In [25]:
df[['va_min_attr_auroc', 'hparams_seed', 'seed']].sort_values(
    ['va_min_attr_auroc', 'hparams_seed'],
    ascending=[False, False] 
)


,va_min_attr_auroc,hparams_seed,seed
20,0.816063,8,2
15,0.815282,8,0
29,0.812421,0,0
25,0.811064,0,2
1,0.810588,7,0
3,0.810579,6,1
13,0.810308,1,0
28,0.809665,7,2
10,0.809105,8,1
26,0.808928,7,1


In [ ]:
#optimal_thresholds

In [ ]:
#best models según va_min_attr_auroc

In [10]:
best_models = df.groupby(['dataset', 'task', 'attr', 'algorithm']).apply(lambda x: x.loc[x['va_min_attr_auroc'].idxmax()])

/tmp/ipykernel_2821546/3613221529.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models = df.groupby(['dataset', 'task', 'attr', 'algorithm']).apply(lambda x: x.loc[x['va_min_attr_auroc'].idxmax()])


In [11]:
best_models

,,,20,store_name,dataset,task,attr,group_def,algorithm,output_dir,data_dir,hparams,hparams_seed,...,heat,use_masks,loadCuts,tuneCut,only_tune_cut,va_auroc,va_min_attr_auroc,te_auroc,va_y,va_preds
dataset,task,attr,algorithm,,,,,,,,,,,,,,,,,,,,,
MIMIC,No Finding,sex,ISNetDANN,e71e3c622a942657993e10cf0b5e652b,MIMIC,No Finding,sex,group,ISNetDANN,/home/lchanch/model_training/ISNetDANN/train/g...,/home/lchanch/df_construction_mapping/image_ma...,{},8,...,False,True,False,False,False,0.829996,0.816063,0.832509,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.6897757, 0.74867094, 0.051150978, 0.1009765..."


In [22]:
best_models[['store_name','hparams_seed', 'seed', 'va_min_attr_auroc']]

,,,20,store_name,hparams_seed,seed,va_min_attr_auroc
dataset,task,attr,algorithm,,,,
MIMIC,No Finding,sex,ISNetDANN,e71e3c622a942657993e10cf0b5e652b,8,2,0.816063


In [17]:
opt_thres = {}
for idx, row in tqdm(best_models.iterrows(), total = len(best_models)):
    dataset, task, attr, algorithm = idx
#     if dataset not in opt_thres:
#         opt_thres[dataset] = {}
    opt_thres[(dataset, task, attr, algorithm)] = np.round(compute_opt_thres(row['va_y'], row['va_preds']), 3)

100%|██████████| 1/1 [00:04<00:00,  4.01s/it]


In [26]:
opt_thres

{('MIMIC', 'No Finding', 'sex', 'ISNetDANN'): 0.44}

In [27]:
pickle.dump(opt_thres, open('opt_thres_sex_ISNetDANN.pkl', 'wb'))

In [29]:
df[['store_name','hparams_seed', 'seed', 'va_min_attr_auroc']].sort_values('hparams_seed')

,store_name,hparams_seed,seed,va_min_attr_auroc
0,b27dd33301c5fb5687a02dd030985e82,0,1,0.785944
25,44c4287cb22a3ad75949f2f0c08ac760,0,2,0.811064
29,f2714f6a5f6986f778afaa995b5471ca,0,0,0.812421
19,ca231313a631bb963708c2f596ad71d6,1,1,0.808161
13,f9a89da122ddabcf330aa7029f7cb068,1,0,0.810308
27,57adf80e204f37d5d86b10528a089726,1,2,0.793833
5,68cc8fadb0a7c92e31b03243b58e9016,2,1,0.749230
22,04e51b215e7e1f28a80137cd4ee51d6d,2,2,0.751368
4,e26c7dfc2d5a55375aa5cb565b50bd31,3,0,0.778468
6,f9effb3951c9786349d51e64d1510997,3,2,0.776380
